# Multidimensional Scaling in Hyperbolic Space

In this notebook, I begin my attempt to implement a Multidimensional Scaling (MDS) algorithm that embeds data into hyperbolic space instead of the target Euclidean space for standard MDS. Just like traditional MDS, this variant h-MDS (hyperbolic MDS) will seek to produce accurate spatial representations of possibly high dimensional datasets, by  attempting to reflect the original pairwise similarities by the corresponding pairwise distances of the output configuration in the embedding space. <br>
<br>
Relevant papers that discuss hyperbolic MDS and outline some general implementation details: <br>
<br>
1. A. Cvetkovski and M. Crovella. Multidimensional scaling in the Poincaré disk. arXiv preprint, arXiv:1105.5332, 2011.
2. Jörg A. Walter and Helge Ritter. 2002. On interactive visualization of high-dimensional data using the hyperbolic plane. In Proceedings of the eighth ACM SIGKDD international conference on Knowledge discovery and data mining (KDD '02). ACM, New York, NY, USA, 123-132. DOI=http://dx.doi.org/10.1145/775047.775065

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import check_random_state
plt.style.use('classic')
plt.rcParams['figure.figsize'] = (6.5, 6.5)
plt.rcParams['figure.facecolor'] = 'white'
from jupyterthemes import jtplot
#jtplot.style()

# ignore warnings
import warnings
warnings.filterwarnings('ignore');

# display multiple outputs within a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

# Intro and key concepts

We use the Poincaré disk as our model of hyperbolic space, and the only fundamental change with respect to MDS in Euclidean space is that we need to use the Poincaré distance metric instead of the Euclidean distance.

We can define the Poincare disk based on the complex plane: $\mathbb{D}=\{z \in \mathbb{C}| | z |<1\}$. We have the symmetric distance function for $\boldsymbol{u}, \boldsymbol{v} \in \mathcal{D}$:

$$
d(\boldsymbol{u}, \boldsymbol{v})=\operatorname{arcosh}\left(1+2 \frac{\|\boldsymbol{u}-\boldsymbol{v}\|^{2}}{\left(1-\|\boldsymbol{u}\|^{2}\right)\left(1-\|\boldsymbol{v}\|^{2}\right)}\right)
$$

The idea of Mobius transformations will also be important, and these are transformations that preserve hyperbolic distances. The Mobius transformations $T$ have the following form: <br> 
<br>
$$
T(z)=\frac{a z+b}{\overline{b} z+\overline{a}}, a, b \in \mathbb{C},|a|^{2}-|b|^{2} \neq 0
$$

Our h-MDS algorithm will have a relatively simple design based on iterative steepest descent and an approximate line search function. Therefore, we need to be able to describe hyperbolic lines based on a given direction $\gamma$ and a starting point $z_0$. For $z_{0} \in \mathbb{D}, \gamma \in \mathbb{C}$ with $|\gamma|=1,$ and $s \geq 0$, this hyperbolic line is given by:<br>
<br>
$$
z_{0}^{\prime}=\frac{\gamma \tanh \frac{s}{2}+z_{0}}{\overline{z_{0}} \gamma \tanh \frac{s}{2}+1}
$$ <br>
where $s$ gives the length along the line and $d_{\mathrm{D}}\left(z_{0}, z_{0}^{\prime}\right)=s$

# Gradient descent and hyperbolic line search

## Descent method

The steepest descent will be based on the gradient of an objective function $E_t(\mathbf{z}, \Delta, \mathbf{W}, \mathbf{I})$ that is defined by these inputs and which gives the embedding error or loss at some given iteration $t$<br>
<br>
One common embedding error is given by the following function defined as the sum of relative squared differences between the initial dissimilarities and the distances in the embedding space:. <br>
<br>
$$
E_{t}(\mathbf{z}, \Delta, \mathbf{W}, \mathbf{I})=\sum_{j=1}^{n} \sum_{k=j+1}^{n} w_{j k} I_{j k}\left(\frac{d_{j k}(t)-\delta_{j k}}{\delta_{j k}}\right)^{2}
$$

The descent direction is defined by the gradient of this error function: <br>
<br>
$$
\mathbf{g}=\nabla E \stackrel{\mathrm{def}}{=}\left[\begin{array}{c}{\frac{\partial E}{\partial y_{1,1}}+i \frac{\partial E}{\partial y_{1,2}}} \\ {\frac{\partial E}{\partial y_{2,1}}+i \frac{\partial E}{\partial y_{2,2}}} \\ {\vdots} \\ {\frac{\partial E}{\partial y_{n, 1}}+i \frac{\partial E}{\partial y_{n, 2}}}\end{array}\right]=\left[\begin{array}{c}{g_{1}} \\ {g_{2}} \\ {\vdots} \\ {g_{n}}\end{array}\right]
$$<br>
We have descent directions $-g_1, \ldots , -g_n$ and the new configuration after the update will have points <br>
<br>
$$
z_{j}^{\prime}=\frac{-r g_{j}+z_{j}}{-r g_{j} \overline{z_{j}}+1}
$$ <br>
and $r \geq 0$ is the step-size parameter

![title](images/pd_mds5.png)

## Approximate Hyperbolic Line Search

We have some more notation and parameters related to line search. $s_M$ sets the window for distance between successive configurations or updates. We need this to be finite to limit points from approaching the boundary at infinite distances. If $s_j$ are distances for updates: $s_{M}=\max _{j} s_{j}<\infty$<br>
<br>
The parameter $r_M$ sets the window for the step size $r$ and we restrict our line search for each update within this window. We have the constraint: $r<\frac{1}{\|\mathbf{g}\|_{\infty}}$ and the following equation that relates these parameters: <br>

$$
r_{M}=\frac{1}{\|\mathbf{g}\|_{\infty}} \cdot \tanh \frac{s_{M}}{2}<\frac{1}{\|\mathbf{g}\|_{\infty}}
$$

During our line search, it would be ideal to find the step size $r$ along the hyperbolic line (but within the window $r_M$) that minimizes the error function. This turns out to be too computationally expensive, and so we just set a sufficient stopping condition and accept any step size that meets the condition. 

To express the stopping condition, we define the error in terms of the step size and let this function be $q(r)$ <br>
<br>
We also define this 'roof' function: $\lambda(r)=q(0)+p \cdot q^{\prime}(0) \cdot r, \quad 0<p<1$ <br>
<br>
and set the condition for acceptable values of step size: $q(r)<\lambda(r), r \in\left(0, r_{M}\right]$ 

![title](images/line_search.png)

We need to find explicit formulas for $q(r)$ and its derivative $q'(r)$ in order to calculate these values and apply the stopping conditions in the approximate line search. <br>
<br>
We can write the updates for each $z_j$ based on the Mobius transformations: $M_{j}(r)=\frac{-r g_{j}+z_{j}}{-r g_{j} \overline{z}_{j}+1}$

The expression for the error in terms of step size can now be written as $q(r)=E(\mathbf{M}(-r \mathbf{g}, \mathbf{z}), \mathbf{\Delta}, \mathbf{W}, \mathbf{I})$ <br>
<br>
$q'(r)$ has the following form: <br>
<br>
$$
\begin{aligned} q^{\prime}(r) &=\frac{d}{d r} q(r)=\\ &=\left(\operatorname{Re} \mathbf{M}^{\prime}(-r \mathbf{g}, \mathbf{z})\right)^{T} \operatorname{Re} \nabla E(\mathbf{M}(-r \mathbf{g}, \mathbf{z}), \boldsymbol{\Delta}, \mathbf{W}, \mathbf{I}) \\ &+\left(\operatorname{Im} \mathbf{M}^{\prime}(-r \mathbf{g}, \mathbf{z})\right)^{T} \operatorname{Im} \nabla E(\mathbf{M}(-r \mathbf{g}, \mathbf{z}), \boldsymbol{\Delta}, \mathbf{W}, \mathbf{I}) \end{aligned}
$$

and this can be explicitly calculated using the following equation: <br>
<br>
$$
M_{j}^{\prime}(r)=\frac{d}{d r} M_{j}(r)=g_{j} \frac{\left|z_{j}\right|^{2}-1}{\left(1-r g_{j} \overline{z_{j}}\right)^{2}}
$$

![title](images/hyp_line.png)

# Implementing h-MDS

For the initial implementation of h-MDS let the error function be given by the Absolute Differences Squared (ADS) <br>
<br>
$$
E=\sum_{j=1}^{n} \sum_{k=j+1}^{n} w_{j k}\left(I_{j k}\left(d_{j k}-a \delta_{j k}\right)\right)^{2}
$$

In [1]:
from IPython.core.display import HTML

def css_styling():
    styles = open("custom_style.css", "r").read()
    return HTML(styles)
css_styling()